In [ ]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk

In [2]:
clean_mini = pd.read_csv('clean_python_stack_052620.csv')

In [3]:
clean_mini.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text
0,multiple json objects into a list,"{""Id"": 1,""product"": ""Mobile"",""price"": 32889,""d...","['python', 'json']",diveinsky,0.0,1.0,2019-08-02,multiple json object list
1,cannot read in image in colour in opencv python,I have just started to use Opencv using python...,"['python-2.7', 'opencv']",Yogesh,0.0,2.0,2019-08-02,can not read image colour opencv python
2,How to resolve related foreign key at insert t...,Motivations\n\nI have data coming from a sourc...,"['python', 'pandas', 'orm', 'sqlalchemy', 'for...",jlandercy,4.0,2.0,2019-08-02,resolve relate foreign key insert time use sql...
3,syntax error in python shorthand expression,I am using bs4 to scrape a website I have thi...,"['python', 'python-3.x', 'beautifulsoup']",QHarr,2.0,2.0,2019-08-02,syntax error python shorthand expression
4,When do we need to pip install future [duplicate],As per the documentation the module future pro...,"['python', 'future']",Jay Joshi,0.0,0.0,2019-08-02,need pip install future duplicate


In [4]:
clean_mini.shape

(1464969, 8)

In [5]:
clean_mini['clean_text'].isna().sum()

12

In [6]:
clean_mini = clean_mini.dropna(subset=['clean_text'])

In [7]:
clean_mini['clean_text'] = clean_mini['clean_text'].str.split(' ')

In [29]:
clean_mini['clean_text'].head()

0                       [multiple, json, object, list]
1      [can, not, read, image, colour, opencv, python]
2    [resolve, relate, foreign, key, insert, time, ...
3       [syntax, error, python, shorthand, expression]
4              [need, pip, install, future, duplicate]
Name: clean_text, dtype: object

In [8]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [9]:
dictionary, corpus = prep_corpus(clean_mini['clean_text'])

Building dictionary...
Building corpus...


In [10]:
MmCorpus.serialize('models/python_corpus.mm', corpus)
dictionary.save('models/python_dict.dict')

In [30]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15)
                                      
lda.save('models/python_lda.model')

CPU times: user 6min 46s, sys: 6.9 s, total: 6min 53s
Wall time: 7min 7s


In [ ]:
#LdaModel.load('models/python_lda.model')

In [31]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [32]:
%%time
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

KeyboardInterrupt: 

In [ ]:
#get_document_topics(bow, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)

In [23]:
from gensim.models import LdaModel

In [33]:
doc_topics = lda.get_document_topics(bow = corpus[0])

In [34]:
doc_topics

[(0, 0.013333494),
 (1, 0.0133335125),
 (2, 0.213332),
 (3, 0.21333477),
 (4, 0.013333494),
 (5, 0.013333494),
 (6, 0.013333494),
 (7, 0.013333494),
 (8, 0.21333066),
 (9, 0.013333494),
 (10, 0.013333513),
 (11, 0.013333494),
 (12, 0.21333407),
 (13, 0.013333494),
 (14, 0.013333511)]